<a href="https://colab.research.google.com/github/arjundvn24/matlab24/blob/main/fSahaj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('./content/USA_Housing.csv')
df

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05
...,...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,4.02,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,2.13,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,5.44,42625.62016,1.198657e+06


In [ ]:
X=df.iloc[:, 0:5].copy()
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947
...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103
4996,78491.27543,6.999135,6.576763,4.02,25616.11549
4997,63390.68689,7.250591,4.805081,2.13,33266.14549
4998,68001.33124,5.534388,7.130144,5.44,42625.62016


In [9]:
Y=df.iloc[:, 5].copy()
Y

0       1.059034e+06
1       1.505891e+06
2       1.058988e+06
3       1.260617e+06
4       6.309435e+05
            ...     
4995    1.060194e+06
4996    1.482618e+06
4997    1.030730e+06
4998    1.198657e+06
4999    1.298950e+06
Name: Price, Length: 5000, dtype: float64

In [ ]:
ss=StandardScaler()
Xstd=ss.fit_transform(X)
X['Avg. Area Income']=Xstd[:, 0]
X['Area Population']=Xstd[:, 4]
X

In [6]:
kf=KFold(n_splits=5, random_state=None)

Beta_collection=[]
R2_collection=[]

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    Y_train , Y_test = Y[train_index] , Y[test_index]


    #Prepend X_test with a column of 1s
    no_of_rows, no_of_columns=X_test.shape
    X_ones=np.ones((no_of_rows, 1))
    X_test_with_ones=np.hstack((X_ones, X_test))


    #Prepend X_train with a column of 1s
    no_of_rows, no_of_columns=X_train.shape
    X_ones=np.ones((no_of_rows, 1))
    X_train_with_ones=np.hstack((X_ones, X_train))

    #Least Squares Method
    X_transpose=np.transpose(X_train_with_ones)
    X_transpose_X=np.matmul(X_transpose,X_train_with_ones)
    X_transpose_X_inverse=np.linalg.inv(X_transpose_X)
    X_transpose_X_inverse_X=np.matmul( X_transpose_X_inverse, X_transpose)
    B=np.matmul( X_transpose_X_inverse_X,Y_train)

    Y_predicted=np.matmul(X_test_with_ones,B)

    score=r2_score(Y_test, Y_predicted)

    #Store all beta matrices and R2 scores
    Beta_collection.append(B)
    R2_collection.append(score)

    

In [7]:
#Select Beta with max R2 score
R2_max_index=R2_collection.index(max(R2_collection))
B=Beta_collection[R2_max_index]
print(f"Best R2 Score={max(R2_collection)}")
print(f"Corresponding beta matrix:\n{B}")

Best R2 Score=0.920850383697766
Corresponding beta matrix:
[-596076.10240762  229921.55808488  165957.41875628  119055.00734636
     911.05670455  151317.80239611]


In [8]:
#Test for 30% data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)


#Prepend X_test with a column of 1s
no_of_rows, no_of_columns=X_test.shape
X_ones=np.ones((no_of_rows, 1))
X_test_with_ones=np.hstack((X_ones, X_test))

Y_predicted=np.matmul(X_test_with_ones,B)
score=r2_score(Y_test, Y_predicted)

print(f"For 30% of data, R2 Score={score}")

For 30% of data, R2 Score=0.9200994781500813
